In [6]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
S = ticker.history(period="1d")["Close"].iloc[-1]
print("Spot AAPL:", S)

Spot AAPL: 260.73040771484375


In [78]:
# Avoir le tx sans risque 
# Pour US treasuries: ^IRX -> 3M  ;  ^FVX -> 5Y  ;  ^TNX -> 10Y  . ^TYX -> 30Y

import yfinance as yf

ticker = yf.Ticker("^IRX")  # T-bill 13 semaines
rate = ticker.history(period="1d")["Close"].iloc[-1] / 100

print(rate)


0.03570000171661377


In [36]:
# Avoir la vol implicite
options = ticker.option_chain()
calls = options.calls

# Exemple : prendre la vol implicite du call ATM le plus proche
atm_call = calls.iloc[(calls['strike'] - S).abs().argsort()[:1]]
sigma = atm_call['impliedVolatility'].values[0]
print(calls)
print(sigma)

         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   AAPL260130C00120000 2026-01-22 16:57:40+00:00   120.0     129.85  139.30   
1   AAPL260130C00125000 2026-01-23 16:11:00+00:00   125.0     124.22  134.35   
2   AAPL260130C00130000 2026-01-23 18:24:44+00:00   130.0     118.11  129.25   
3   AAPL260130C00135000 2026-01-22 18:06:52+00:00   135.0     115.05  124.25   
4   AAPL260130C00145000 2026-01-22 18:23:55+00:00   145.0     105.35  114.30   
5   AAPL260130C00150000 2026-01-26 18:51:31+00:00   150.0     105.15  109.30   
6   AAPL260130C00155000 2026-01-09 14:46:55+00:00   155.0     103.73  104.30   
7   AAPL260130C00170000 2026-01-23 17:42:28+00:00   170.0      78.55   89.30   
8   AAPL260130C00175000 2026-01-22 17:21:16+00:00   175.0      74.97   84.35   
9   AAPL260130C00180000 2026-01-21 15:23:33+00:00   180.0      67.27   79.30   
10  AAPL260130C00185000 2026-01-23 15:01:08+00:00   185.0      61.11   74.30   
11  AAPL260130C00190000 2026-01-27 15:29

In [71]:
from scipy.stats import norm
import numpy as np

def BS(S, K, T, r, q, sigma, option_type="call"):

    d1 = (np.log(S / K) + (r-q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        price = S * np.exp(-q * T) *norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S* np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")

    return price

In [77]:
call_price=BS(S,255,1,rate,0.05,sigma,"call")
put_price=BS(S,280,1,rate,0.03,sigma,"put")

print("Call:", call_price)
print("Put:", put_price)

Call: 49.72981828809455
Put: 60.632020010110935
